In [ ]:
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt 
import matplotlib as mpl
import seaborn as sns 
import geopandas as gpd
import pandas as pd 
import numpy as np
import requests
import shapely
import pickle
import progressbar
import geopy.distance
import geopandas as gpd
import pyproj
import os
import matplotlib.cm as cm
import matplotlib.colors as colors
import itertools
import warnings
import random
import timeit
import time
import threading
import functions
from joblib import Parallel, delayed
from funcs_a import *
from shapely.geometry import Point, Polygon, LineString 
from shapely.ops import nearest_points
from shapely.ops import transform
from shapely import geometry, ops
from functools import partial
from itertools import combinations
from multiprocessing import Pool
from random import choice, randint
from scipy.sparse import lil_matrix
warnings.simplefilter('ignore')

In [2]:
# Load graph
with open(r"C:\Users\anton\Desktop\Master-Oppgave\Kode\New_Traffic\data\BaseGraph_E_NOR_wagrades.pickle", 'rb') as file:
    G = pickle.load(file)

In [3]:
#Read OD Nodes and All current ChargingStationsNodes
OD = pd.read_csv(r"C:\Users\anton\Desktop\Master-Oppgave\Kode\apert\data\OD_new.csv", encoding="Cp1252")
OD = OD.drop('Unnamed: 0', axis=1)
# OD_test = pd.read_csv("Data/OD_test.csv", encoding="iso8859_10")
# OD_test = OD_test.drop('Unnamed: 0', axis=1)
CS = pd.read_csv(r"C:\Users\anton\Desktop\Master-Oppgave\Kode\apert\data\AllCurrentCS.csv", encoding="Cp1252")
CS = CS.drop('Unnamed: 0', axis=1)

In [40]:
graph_attrs = {'crs': 'epsg:4326', 'simplified': False}



In [4]:
H = G.copy()
H = H.to_undirected()

In [5]:
#Get nearest node to each O-D location
lat = list(OD.lat)
lon = list(OD.lon)
ODs= ox.distance.nearest_nodes(H, lon, lat)
OD['node'] = ODs

#Get nearest node to each CS
latA = list(CS.Latitude)
lonA = list(CS.Longitude)
AllCSnodes = ox.distance.nearest_nodes(H, lonA, latA)
CS['node']=AllCSnodes
AllCSnodes1 = list(CS.node)
#Create combinations of O-D pairs. Used for several functions later on
combos = combinations(ODs,2)
OD_pairs = []
for i in combos:
    OD_pairs.append(list(i))

In [6]:
def process_drivetime(drivetime_values):
    sum_drivetime = 0
    for drivetime in drivetime_values:
        if drivetime != -1:
            sum_drivetime += drivetime
    return sum_drivetime

def process_length(length_values):
    sum_length = 0
    for length in length_values:
        if length != -1:
            sum_length += length
    return sum_length


def get_edge_attributes(cumlist):
    
    
    """
    Function that takes a list of network nodes and merge their attributes as the different attributes need to be treated and formatted differently. This specifically applies for LineString.
    
    Input:
    cumlist: Cummulative list of nodes
    
    """

    cumlength = len(cumlist)
    path_attributes = dict()
    attrs_to_sum = {"length", "drivetime",}
    attrs_to_set = {'oneway', 'ref', 'name', 'funcroadclass', 'roadclass'}
    dummy_attrs = {'isFerry', 'isTunnel', 'isBridge'}
    nodes_to_remove = []
    
    for i in range(cumlength-1):
        u = cumlist[i]
        v = cumlist[i+1]
        edge_data = H.edges[u, v, 0]

        for attr in edge_data:
            if attr in path_attributes:
            # if this key already exists in the dict, append it to the value list
                path_attributes[attr].append(edge_data[attr])
            else:
            # if this key doesn't already exist, set the value to a list containing the one value
                path_attributes[attr] = [edge_data[attr]]

    for attr in path_attributes:
        if attr == 'grade' or attr == 'length_weight':
            path_attributes[attr] = list(path_attributes[attr]) 
        elif attr in dummy_attrs:
            #If attribute is isFerry, isBridge og isTunnel, returns the value for first and last edge as these are candidate locations.
            if type(path_attributes[attr][0]) == list and type(path_attributes[attr][-1]) == list:
                fromnode = path_attributes[attr][0][0]
                tonode = path_attributes[attr][-1][-1]
                
            elif type(path_attributes[attr][0]) != list and type(path_attributes[attr][-1]) != list:
                fromnode = path_attributes[attr][0]
                tonode = path_attributes[attr][-1]
                
            elif type(path_attributes[attr][0]) == list and type(path_attributes[attr][-1]) != list:
                fromnode = path_attributes[attr][0][0]
                tonode = path_attributes[attr][-1]
                
            elif type(path_attributes[attr][0]) != list and type(path_attributes[attr][-1]) == list:
                fromnode = path_attributes[attr][0]
                tonode = path_attributes[attr][-1][-1]
            path_attributes[attr] = list((fromnode, tonode))

        elif attr in attrs_to_sum:
            if attr == 'drivetime' or attr == 'length':
                try:
                    flattened_values = []
                    for value in path_attributes[attr]:
                        if isinstance(value, (int, float)):
                            if value != -1:
                              flattened_values.append(value)
                        elif isinstance(value, list):
                            flattened_values.extend([v for v in value if v != -1])
                    if attr == 'drivetime':
                        path_attributes[attr] = process_drivetime(flattened_values)
                    elif attr == 'length':
                        path_attributes[attr] = process_length(flattened_values)
                except TypeError:
                    print(f"Error summing attribute '{attr}', value: {path_attributes[attr]}")

        elif attr == 'id':
            path_attributes[attr] = list((cumlist[0],cumlist[-1]))

        
        elif attr == 'geometry':
                path_attributes[attr] = ops.linemerge(path_attributes[attr])
                
        elif attr == 'oneway':
            path_attributes[attr] = 'False'


        else:
        # otherwise, if there are multiple values, keep one of each
            try:
                path_attributes[attr] = list(set(path_attributes[attr]))
            except TypeError:
                try:
                    path_attributes[attr] = list(set(path_attributes[attr][0]))
                except:
                    path_attributes[attr] = path_attributes[attr]
            
    path_attributes['artificial'] = 1
    return path_attributes

In [7]:
def shorten_edges_v2(G, OD, cutoff):
    
    """
    Simplifies the edges in the network in order to make the runtimes of the GA/greedy substitution feassible. Depending on the threshold value, the function will remove all nodes/edges within a certain range, store and merge
    their attributes, before creating a new edge between the first and last node in the specified range. If cutoff is for example 50000, the function will remove all nodes except the first and last node for every 50th km and 
    create a new edge between the first and last node. This i then repeated for each path between each OD pair.
    
    There are also certain statements to make sure that the same nodes are used if several paths intersect. This usually happens when multiple routes shares the same path. 
    
    Input:
    G: Networkx Multigraph
    OD: CSV file with coordinates of each OD location in Norway
    Threshold: Cutoff in meters at which edges at merged. 
    """

    
    #Create flat list of O-D pairs
    OD_pairs_flat = [item for sublist in OD_pairs for item in sublist]
    
    #Create empty MultiGraph
    B = nx.MultiGraph(crs='epsg:4326')
    
    #Initialize progressbar to monitor progress of function while running.
    maxval = len(OD_pairs)
    bar    = progressbar.ProgressBar(maxval=maxval, \
        widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])

    counter=0
    bar.start()

    UsedNodes = []
    for i in OD_pairs:
        fromOD         = i[0]
        toOD           = i[1]
        path           = nx.shortest_path(H, fromOD, toOD, 'length')
        pathlength     = len(path)
        lengthofpath   = nx.shortest_path_length(H, fromOD, toOD, 'length')
        cumlength      = 0
        cumlist        = []
        cutoff         = cutoff


        #for each in edge in path[i]
        for i in range(pathlength-1):
            fromnode = path[i]
            tonode = path[i+1]
            templength = H.edges[fromnode, tonode, 0]['length']

            cumlist.append(fromnode)
            cumlist.append(tonode)
            cumlength += templength

            #First check if tonode already exists in B. If it does, it means that a previously created path has used same route. 
            #In this case we want to use existing nodes on path rather than creating new ones.
            if tonode in UsedNodes:
                #If edge does not exist, then add it
                if B.has_edge(cumlist[0], cumlist[-1]) is False:
                    cumlist = list(dict.fromkeys(cumlist))
                    #Get attributes for nodes and edge to add
                    fnodeattr = H.nodes[cumlist[0]]
                    tnodeattr = H.nodes[cumlist[-1]]
                    edgeattr = get_edge_attributes(cumlist)

                    #Add nodes and edge
                    B.add_node(cumlist[0], **fnodeattr)
                    B.add_node(cumlist[-1], **tnodeattr)
                    B.add_edge(cumlist[0], cumlist[-1], **edgeattr)

                    #Added nodes are appended to all UsedNodes
                    UsedNodes.append(cumlist[0])
                    UsedNodes.append(cumlist[-1])
                    cumlist = []
                    cumlength = 0


                #Else, skip this part, but empty cummulative path and length
                else:
                    cumlist = []
                    cumlength = 0
                    pass
                
            elif tonode in AllCSnodes: #if next node is a CS node
                #If edge does not exist, then add it
                if B.has_edge(cumlist[0], cumlist[-1]) is False:
                    cumlist = list(dict.fromkeys(cumlist))
                    #Get attributes for nodes and edge to add
                    fnodeattr = H.nodes[cumlist[0]]
                    tnodeattr = H.nodes[cumlist[-1]]
                    edgeattr = get_edge_attributes(cumlist)

                    #Add nodes and edge
                    B.add_node(cumlist[0], **fnodeattr)
                    B.add_node(cumlist[-1], **tnodeattr)
                    B.add_edge(cumlist[0], cumlist[-1], **edgeattr)

                    #Added nodes are appended to all UsedNodes
                    UsedNodes.append(cumlist[0])
                    UsedNodes.append(cumlist[-1])
                    cumlist = []
                    cumlength = 0


                #Else, skip this part, but empty cummulative path and length
                else:
                    cumlist = []
                    cumlength = 0
                    pass
                

            #Next, check if tonode is an OD node. If it is we will create edge from start of cumlist to the OD node.
            elif tonode in OD_pairs_flat:
                #If edge does not exist, then add it
                if B.has_edge(cumlist[0], cumlist[-1]) is False:
                    cumlist = list(dict.fromkeys(cumlist))
                    #Get attributes for nodes and edge to add
                    fnodeattr = H.nodes[cumlist[0]]
                    tnodeattr = H.nodes[cumlist[-1]]
                    edgeattr = get_edge_attributes(cumlist)

                    #Add nodes and edge
                    B.add_node(cumlist[0], **fnodeattr)
                    B.add_node(cumlist[-1], **tnodeattr)
                    B.add_edge(cumlist[0], cumlist[-1], **edgeattr)

                    UsedNodes.append(cumlist[0])
                    UsedNodes.append(cumlist[-1])
                    cumlist = []
                    cumlength = 0
                #Else, skip this part, but empty cummulative path and length
                else:
                    cumlist = []
                    cumlength = 0
                    pass

            #Next, check if the cummulative length in path is higher than the set threshold. If it is we will create a new edge in B
            #Can consider adding an if inside here that check if there is a node within a certain radius, and skip if it is.
            #This could prevent that nodes on completely different paths are very close or that a node is set very close to an OD node. 
            #Would then find last element in cumlist in H and get lat,lon, then use ox nearest node to find nearest node in B and finally use geopy to find distance between them. 

            elif cumlength >= cutoff:            


                if B.has_edge(cumlist[0], cumlist[-1]) is False:
                    #Remove duplicates from cumlist
                    cumlist = list(dict.fromkeys(cumlist))
                    distanceOD = 100000
                    #Before we add new edge, check if destination node is within threshold/2
                    lat = H.nodes[cumlist[-1]]['y']
                    lon = H.nodes[cumlist[-1]]['x']
                    latlon = (lat, lon)
                    latdestination = H.nodes[toOD]['y']
                    londestination = H.nodes[toOD]['x']

                    #Also check if there is an already created node nearby
                    try:
                        potentialnode = ox.distance.nearest_nodes(B, lat, lon)
                        clatclon = (latdestination,londestination)
                        distanceOD = (geopy.distance.geodesic(latlon,clatclon).m)

                    except ValueError:
                        pass

                    # clatclon = (latdestination,londestination)
                    # distanceOD = (geopy.distance.geodesic(latlon,clatclon).m)
                    # distancepath = geopy.distance.geodesic(latlon, platplon).m

                    if distanceOD <= cutoff /2:
                        pass


                    else:
                        #Get attributes for nodes and edge to add
                        fnodeattr = H.nodes[cumlist[0]]
                        tnodeattr = H.nodes[cumlist[-1]]
                        edgeattr = get_edge_attributes(cumlist)

                        #Add nodes and edge
                        B.add_node(cumlist[0], **fnodeattr)
                        B.add_node(cumlist[-1], **tnodeattr)
                        B.add_edge(cumlist[0], cumlist[-1], **edgeattr)

                        UsedNodes.append(cumlist[0])
                        UsedNodes.append(cumlist[-1])                
                        cumlist = []
                        cumlength = 0

                #Else, skip this part, but empty cummulative path and length
                else:
                    cumlist = []
                    cumlength = 0
                    pass

        bar.update(counter)
        counter+=1

    bar.finish()

    #Set OD=1 for all O-D nodes and CS=1 for all current CS nodes. 
    nx.set_node_attributes(B, 0, 'OD')
    nx.set_node_attributes(B, 0, 'CS')

    attrs = {}
    for i in OD_pairs_flat:
        O = i
        attrs[i] = {'OD':1}
    nx.set_node_attributes(B, attrs)
    
    attrs = {}
    for i in AllCSnodes:
        O = i
        attrs[i] = {'CS':1}
    nx.set_node_attributes(B, attrs)
    
    
    print("Complete!")
    return B

In [10]:
B = shorten_edges_v2(H, OD, 10000) #Select cut-off value
nx.write_gpickle(B,r"C:\Users\anton\Desktop\Master-Oppgave\Kode\apert\data\B10000.gpickle")
B = shorten_edges_v2(H, OD, 2000)
nx.write_gpickle(B,r"C:\Users\anton\Desktop\Master-Oppgave\Kode\apert\data\B2000.gpickle")
B = shorten_edges_v2(H, OD, 5000)
nx.write_gpickle(B,r"C:\Users\anton\Desktop\Master-Oppgave\Kode\apert\data\B5000.gpickle")



Complete!


AttributeError: module 'networkx' has no attribute 'write_gpickle'